In [ ]:
!pip install flair

import numpy as np
from flair.embeddings import WordEmbeddings
from flair.embeddings import TransformerWordEmbeddings
from flair.data import Sentence
from scipy.spatial import distance

bert_embedding = TransformerWordEmbeddings('bert-base-multilingual-cased')

import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher
from spacy.tokens import Span

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [ ]:
candidate_sentences = pd.read_csv("/content/wiki_sentences_v11.csv", encoding= 'unicode_escape', index_col="gene")
candidate_sentences

In [ ]:
import numpy as np
import argparse
from scipy import io, spatial
import time
from random import shuffle
import random
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
import scipy.io as sio
import os
import shutil
import random

# Set random seed and other hyperparameters
rand_seed = 42
dataset = '/content/DEFECT'
lr = 0.1
norm_type = 'L2'
early_stop = 20
epochs = 20

# Function to normalize features
def normalizeFeature(x):
    x = x + 1e-10
    feature_norm = np.sum(x**2, axis=1)**0.5  # Compute L2-norm
    feat = x / feature_norm[:, np.newaxis]
    return feat

# Function to update weight matrix W
def update_W(W, idx, train_classes, beta):
    for j in idx:
        X_n = X_train[:, j]
        y_n = labels_train[j]
        y_ = train_classes[train_classes != y_n]
        XW = np.dot(X_n, W)
        gt_class_score = np.dot(XW, train_sig[:, y_n])

        for i in range(len(y_)):
            label = random.choice(y_)
            score = 1 + np.dot(XW, train_sig[:, label]) - gt_class_score  # Margin should always be 1.
            if score > 0:
                Y = np.expand_dims(train_sig[:, y_n] - train_sig[:, label], axis=0)
                W += lr * beta[int(y_.shape[0] / (i + 1))] * np.dot(np.expand_dims(X_n, axis=1), Y)
                break
    return W

# Function to perform training
def fit():
    print('Training...\n')

    best_val_acc = 0.0
    best_tr_acc = 0.0
    best_val_ep = -1
    best_tr_ep = -1

    rand_idx = np.arange(X_train.shape[1])

    W = np.random.rand(X_train.shape[0], train_sig.shape[0])
    W = normalizeFeature(W.T).T

    train_classes = np.unique(labels_train)

    beta = np.zeros(len(train_classes))
    for i in range(1, beta.shape[0]):
        sum_alpha = 0.0
        for j in range(1, i + 1):
            sum_alpha += 1 / j
        beta[i] = sum_alpha

    for ep in range(epochs):
        start = time.time()
        shuffle(rand_idx)
        W = update_W(W, rand_idx, train_classes, beta)

        tr_acc = zsl_acc(X_train, W, labels_train, train_sig)
        val_acc = zsl_acc(X_val, W, labels_val, val_sig)

        end = time.time()
        elapsed = end - start

        print('Epoch:{}; Train Acc:{}; Val Acc:{}; Time taken:{:.0f}m {:.0f}s\n'.format(ep + 1, tr_acc, val_acc, elapsed // 60, elapsed % 60))

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_val_ep = ep + 1
            best_W = np.copy(W)

        if tr_acc > best_tr_acc:
            best_tr_ep = ep + 1
            best_tr_acc = tr_acc

        if ep + 1 - best_val_ep > early_stop:
            print('Early Stopping by {} epochs. Exiting...'.format(epochs - (ep + 1)))
            break

    print('\nBest Val Acc:{} @ Epoch {}. Best Train Acc:{} @ Epoch {}\n'.format(best_val_acc, best_val_ep, best_tr_acc, best_tr_ep))

    return best_W

# Function to calculate zero-shot learning accuracy
def zsl_acc(X, W, y_true, sig):
    XW = np.dot(X.T, W)  # N x k
    dist = 1 - spatial.distance.cdist(XW, sig.T, 'cosine')  # N x C (number of classes)
    predicted_classes = np.array([np.argmax(output) for output in dist])
    cm = confusion_matrix(y_true, predicted_classes)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    acc = sum(cm.diagonal()) / sig.shape[1]
    return acc

# Function to swap rows in an array
def Swap(arr, start_index, last_index):
    arr[[start_index, last_index], :] = arr[[last_index, start_index], :]

# Set random seeds
random.seed(rand_seed)
np.random.seed(rand_seed)

data_folder = dataset + '/'
res101 = io.loadmat(data_folder + 'res101.mat')
att_splits = io.loadmat(data_folder + 'att_splits.mat')

train_loc = 'train_loc'
val_loc = 'val_loc'
test_loc = 'test_unseen_loc'

feat = res101['features']
X_train = feat[:, np.squeeze(att_splits[train_loc] - 1)]
X_val = feat[:, np.squeeze(att_splits[val_loc] - 1)]
X_test = feat[:, np.squeeze(att_splits[test_loc] - 1)]

print('Tr:{}; Val:{}; Ts:{}\n'.format(X_train.shape[1], X_val.shape[1], X_test.shape[1]))

labels = res101['labels']
labels_train = np.squeeze(labels[np.squeeze(att_splits[train_loc] - 1)])
labels_val = np.squeeze(labels[np.squeeze(att_splits[val_loc] - 1)])
labels_test = np.squeeze(labels[np.squeeze(att_splits[test_loc] - 1)])

train_labels_seen = np.unique(labels_train)
val_labels_unseen = np.unique(labels_val)
test_labels_unseen = np.unique(labels_test)

i = 0
for labels in train_labels_seen:
    labels_train[labels_train == labels] = i
    i += 1

j = 0
for labels in val_labels_unseen:
    labels_val[labels_val == labels] = j
    j += 1

k = 0
for labels in test_labels_unseen:
    labels_test[labels_test == labels] = k
    k += 1

# Load attribute data
sig = att_splits['att']
train_sig = sig[:, train_labels_seen - 1]
val_sig = sig[:, val_labels_unseen - 1]
test_sig = sig[:, test_labels_unseen - 1]

if norm_type == 'std':
    print('Standard Scaling\n')
    scaler = preprocessing.StandardScaler()
    scaler.fit(X_train.T)
    X_train = scaler.transform(X_train.T).T
    X_val = scaler.transform(X_val.T).T
    X_test = scaler.transform(X_test.T).T

if norm_type == 'L2':
    print('L2 norm Scaling\n')
    X_train = normalizeFeature(X_train.T).T
    # X_val = normalizeFeature(X_val.T).T
    # X_test = normalizeFeature(X_test.T).T


n = 40
best_ever=0
sucess = 1
failur = 1
sucess_2 = 1
failur_2 = 1
count_2 = 0.25*n
count = 0.1*n
m = 1
m2 = 1



for i in range(n):
  #evaluate()
  best_W = fit()
  print('Testing...\n')
  test_acc = zsl_acc(X_test, best_W, labels_test, test_sig)
  print('Test Acc:{}'.format(test_acc))
  #print(test_acc)
  m = sucess/(sucess + failur)
  m2 = sucess_2/(sucess_2 + failur_2)
  print(m)
  print(m2)
  k = m - random.uniform(0, 1)
  k2 = m2 - random.uniform(0, 1)
  count = count - 1
  count_2 = count_2 - 1
  if count_2 <= 0:
    sucess_2 = 1
    failur_2 = 1
  if count <= 0:
    sucess = 1
    failur = 1




  if 0 <= m2 <= 0.5:
    print('Deep exploration')
    a1 = 1
    b1 = random.randint(41, 43)
    b1 = str(a1) + str(b1)
    b1 = int(b1)
    print(b1)
  scentence_1=[]
  scentence_1=[candidate_sentences['sentence'][a1],candidate_sentences['sentence'][b1]]
  print(scentence_1)
  scentence_1 = ', '.join(scentence_1)
  print(scentence_1)
  scentence_1 = Sentence(scentence_1)
  bert_embedding.embed(scentence_1)
  black_line=np.array(scentence_1[1].embedding)


  if 0 <= m2 <= 0.5:
    print('Deep exploration')
    a2 = 2
    b2 = random.randint(41, 43)
    b2 = str(a2) + str(b2)
    b2 = int(b2)
    print(b2)
  scentence_2=[]
  scentence_2=[candidate_sentences['sentence'][a2],candidate_sentences['sentence'][b2]]
  print(scentence_2)
  scentence_2 = ', '.join(scentence_2)
  print(scentence_2)
  scentence_2 = Sentence(scentence_2)
  bert_embedding.embed(scentence_2)
  black_triangle=np.array(scentence_2[1].embedding)

  if 0 <= m2 <= 0.5:
    print('Deep exploration')
    a3 = 3
    b3 = random.randint(41, 43)
    b3 = str(a3) + str(b3)
    b3 = int(b3)
    print(b3)
  scentence_3=[]
  scentence_3=[candidate_sentences['sentence'][a3],candidate_sentences['sentence'][b3]]
  print(scentence_3)
  scentence_3 = ', '.join(scentence_3)
  print(scentence_3)
  scentence_3 = Sentence(scentence_3)
  bert_embedding.embed(scentence_3)
  black_circle=np.array(scentence_3[1].embedding)


  if 0 <= m2 <= 0.5:
    print('Deep exploration')
    a4 = 4
    b4 = random.randint(41, 43)
    b4 = str(a4) + str(b4)
    b4 = int(b4)
    print(b4)
  scentence_4=[]
  scentence_4=[candidate_sentences['sentence'][a4],candidate_sentences['sentence'][b4]]
  print(scentence_4)
  scentence_4 = ', '.join(scentence_4)
  print(scentence_4)
  scentence_4 = Sentence(scentence_4)
  bert_embedding.embed(scentence_4)
  black_rectangle=np.array(scentence_4[1].embedding)

  if 0 <= m2 <= 0.5:
    print('Deep exploration')
    a5 = 5
    b5 = random.randint(41, 43)
    b5 = str(a5) + str(b5)
    b5 = int(b5)
    print(b5)
  scentence_5=[]
  scentence_5=[candidate_sentences['sentence'][a5],candidate_sentences['sentence'][b5]]
  print(scentence_5)
  scentence_5 = ', '.join(scentence_5)
  print(scentence_5)
  scentence_5 = Sentence(scentence_5)
  bert_embedding.embed(scentence_5)
  white_line =np.array(scentence_5[1].embedding)

  if 0 <= m2 <= 0.5:
    print('Deep exploration')
    a6 = 6
    b6 = random.randint(41, 43)
    b6 = str(a6) + str(b6)
    b6 = int(b6)
    print(b6)
  scentence_6=[]
  scentence_6=[candidate_sentences['sentence'][a6],candidate_sentences['sentence'][b6]]
  print(scentence_6)
  scentence_6 = ', '.join(scentence_6)
  print(scentence_6)
  scentence_6 = Sentence(scentence_6)
  bert_embedding.embed(scentence_6)
  white_triangle =np.array(scentence_6[1].embedding)


  if 0 <= m2 <= 0.5:
    print('Deep exploration')
    a7 = 7
    b7 = random.randint(41, 43)
    b7 = str(a7) + str(b7)
    b7 = int(b7)
    print(b7)
  scentence_7=[]
  scentence_7=[candidate_sentences['sentence'][a7],candidate_sentences['sentence'][b7]]
  print(scentence_7)
  scentence_7 = ', '.join(scentence_7)
  print(scentence_7)
  scentence_7 = Sentence(scentence_7)
  bert_embedding.embed(scentence_7)
  white_circle=np.array(scentence_7[1].embedding)

  if 0 <= m2 <= 0.5:
    print('Deep exploration')
    a8 = 8
    b8 = random.randint(41, 43)
    b8 = str(a8) + str(b8)
    b8 = int(b8)
    print(b8)
  scentence_8=[]
  scentence_8=[candidate_sentences['sentence'][a8],candidate_sentences['sentence'][b8]]
  print(scentence_8)
  scentence_8 = ', '.join(scentence_8)
  print(scentence_8)
  scentence_8 = Sentence(scentence_8)
  bert_embedding.embed(scentence_8)
  white_rectangle =np.array(scentence_8[1].embedding)

  if 0 <= m2 <= 0.5:
    print('Deep exploration')
    a9 = 9
    b9 = random.randint(41, 43)
    b9 = str(a9) + str(b9)
    b9 = int(b9)
    print(b9)
  scentence_9=[]
  scentence_9=[candidate_sentences['sentence'][a9],candidate_sentences['sentence'][b9]]
  print(scentence_9)
  scentence_9 = ', '.join(scentence_9)
  print(scentence_9)
  scentence_9 = Sentence(scentence_9)
  bert_embedding.embed(scentence_9)
  red_line =np.array(scentence_9[1].embedding)


  if 0 <= m2 <= 0.5:
    print('Deep exploration')
    a10 = 10
    b10 = random.randint(41, 43)
    b10 = str(a10) + str(b10)
    b10 = int(b10)
    print(b10)
  scentence_10=[]
  scentence_10=[candidate_sentences['sentence'][a10],candidate_sentences['sentence'][b10]]
  print(scentence_10)
  scentence_10 = ', '.join(scentence_10)
  print(scentence_10)
  scentence_10 = Sentence(scentence_10)
  bert_embedding.embed(scentence_10)
  red_triangle =np.array(scentence_10[1].embedding)


  mat_contents = sio.loadmat('/content/att_splits.mat')

  mat = {}
  mat['att'] = np.vstack([black_line ,black_triangle,black_circle,black_rectangle,white_line,white_triangle,white_circle,white_rectangle,red_line,red_triangle]).transpose()
  mat['train_loc'] = mat_contents['train_loc']
  mat['val_loc'] = mat_contents['val_loc']
  mat['trainval_loc'] = mat_contents['trainval_loc']
  mat['test_seen_loc'] = mat_contents['test_seen_loc']
  mat['test_unseen_loc'] = mat_contents['test_unseen_loc']
  filemat = 'att_splits.mat'
  sio.savemat(filemat, mat)
  mat_contents = sio.loadmat('/content/att_splits.mat')






  if k < 0: #exploraton
    print('exploraton')
    if test_acc >= best_ever:
      shutil.copy("/content/att_splits.mat", "/content/best/att_splits.mat")
      best_ever = test_acc
      sucess = sucess + 1
      sucess_2 = sucess_2 + 1
      mat_contents = sio.loadmat('/content/DEFECT/att_splits.mat')
      sig=mat_contents['att']
      sig = np.take(sig,np.random.permutation(sig.shape[0]),axis=0,out=sig)
      print(sig)
      mat = {}
      mat['att'] = sig
      mat['train_loc'] = att_splits['train_loc']
      mat['val_loc'] = att_splits['val_loc']
      mat['trainval_loc'] = att_splits['trainval_loc']
      mat['test_seen_loc'] = att_splits['test_seen_loc']
      mat['test_unseen_loc'] = att_splits['test_unseen_loc']
      filemat = 'att_splits.mat'
      sio.savemat(filemat, mat)
      shutil.copy("/content/att_splits.mat", "/content/DEFECT/att_splits.mat")
    elif test_acc < best_ever:
      failur = failur + 1
      failur_2 = failur_2 + 1
      mat_contents = sio.loadmat('/content/best/att_splits.mat')
      sig=mat_contents['att']
      sig = np.take(sig,np.random.permutation(sig.shape[0]),axis=0,out=sig)
      print(sig)
      mat = {}
      mat['att'] = sig
      mat['train_loc'] = att_splits['train_loc']
      mat['val_loc'] = att_splits['val_loc']
      mat['trainval_loc'] = att_splits['trainval_loc']
      mat['test_seen_loc'] = att_splits['test_seen_loc']
      mat['test_unseen_loc'] = att_splits['test_unseen_loc']
      filemat = 'att_splits.mat'
      sio.savemat(filemat, mat)
      shutil.copy("/content/att_splits.mat", "/content/DEFECT/att_splits.mat")
  if k >= 0: #exploitation
    print('exploitation')
    if test_acc >= best_ever:
      shutil.copy("/content/DEFECT/att_splits.mat", "/content/best/att_splits.mat")
      best_ever = test_acc
      sucess = sucess + 1
      sucess_2 = sucess_2 + 1
      mat_contents = sio.loadmat('/content/DEFECT/att_splits.mat')
      sig=mat_contents['att']
      x= random.randint(0, 384)
      y= random.randint(385, 768)
      Swap(sig, x,y)
      print(sig)
      mat = {}
      mat['att'] = sig
      mat['train_loc'] = att_splits['train_loc']
      mat['val_loc'] = att_splits['val_loc']
      mat['trainval_loc'] = att_splits['trainval_loc']
      mat['test_seen_loc'] = att_splits['test_seen_loc']
      mat['test_unseen_loc'] = att_splits['test_unseen_loc']
      filemat = 'att_splits.mat'
      sio.savemat(filemat, mat)
      shutil.copy("/content/att_splits.mat", "/content/DEFECT/att_splits.mat")
    elif test_acc < best_ever:
      failur = failur + 1
      failur_2 = failur_2 + 1
      mat_contents = sio.loadmat('/content/best/att_splits.mat')
      sig=mat_contents['att']
      x= random.randint(0, 384)
      y= random.randint(385, 768)
      Swap(sig, x,y)
      print(sig)
      mat = {}
      mat['att'] = sig
      mat['train_loc'] = att_splits['train_loc']
      mat['val_loc'] = att_splits['val_loc']
      mat['trainval_loc'] = att_splits['trainval_loc']
      mat['test_seen_loc'] = att_splits['test_seen_loc']
      mat['test_unseen_loc'] = att_splits['test_unseen_loc']
      filemat = 'att_splits.mat'
      sio.savemat(filemat, mat)
      shutil.copy("/content/att_splits.mat", "/content/DEFECT/att_splits.mat")


# Play an audio beep. Any audio URL will do.
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')



In [ ]:
#ZSL 3 unseen classes
best_ever